In [1]:
from Bio import SeqIO

In [2]:
def hash_kmer(kmer, prime=31):
    hash_value = 0
    for char in kmer:
        hash_value = (hash_value * prime + ord(char)) % len(sequences)
    return hash_value

In [3]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_score=-2):
    # Needleman-Wunsch algorithm for global alignment
    m, n = len(seq1), len(seq2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        dp[i][0] = dp[i - 1][0] + gap_score

    for j in range(1, n + 1):
        dp[0][j] = dp[0][j - 1] + gap_score

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = dp[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = dp[i - 1][j] + gap_score
            insert = dp[i][j - 1] + gap_score
            dp[i][j] = max(match, delete, insert)

    i, j = m, n
    aligned_seq1 = ""
    aligned_seq2 = ""

    while i > 0 and j > 0:
        if seq1[i - 1] == seq2[j - 1]:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif dp[i][j] == dp[i - 1][j] + gap_score:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1

    while i > 0:
        aligned_seq1 = seq1[i - 1] + aligned_seq1
        aligned_seq2 = "-" + aligned_seq2
        i -= 1

    while j > 0:
        aligned_seq1 = "-" + aligned_seq1
        aligned_seq2 = seq2[j - 1] + aligned_seq2
        j -= 1

    return aligned_seq1, aligned_seq2, dp[m][n]


In [4]:
def find_matching_kmers(sequence, kmer_length, hash_map):
    matching_kmers = []
    for i in range(len(sequence) - kmer_length + 1):
        kmer = sequence[i:i + kmer_length]
        hash_value = hash_kmer(kmer)
        if hash_value in hash_map:
            matching_kmers.extend(hash_map[hash_value])
    return matching_kmers


In [5]:
fasta_file = "fasta_file.fasta"
reference_sequence_id = ">NC_000016.10:176680-177522 Homo sapiens chromosome 16, GRCh38.p14 Primary Assembly"

In [6]:
sequences = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

reference_sequence = sequences.get(reference_sequence_id)
if reference_sequence is None:
    print(f"Reference sequence '{reference_sequence_id}' not found in the FASTA file.")
else:
    del sequences[reference_sequence_id]


FileNotFoundError: [Errno 2] No such file or directory: 'your_fasta_file.fasta'

In [ ]:
kmer_length = 10
hash_map = {}

for sequence_id, sequence in sequences.items():
    for i in range(len(sequence) - kmer_length + 1):
        kmer = sequence[i:i + kmer_length]
        hash_value = hash_kmer(kmer)
        if hash_value in hash_map:
            hash_map[hash_value].append((sequence_id, i))
        else:
            hash_map[hash_value] = [(sequence_id, i)]

print("Hash mapping complete.")

matching_kmers = find_matching_kmers(reference_sequence.seq, kmer_length, hash_map)
print("Matching k-mers found:", matching_kmers)

for sequence_id, _ in matching_kmers:
    seq_to_align = sequences[sequence_id].seq
    print(f"Aligning reference sequence and {sequence_id}...")

    # Needleman-Wunsch alignment
    aligned_ref_seq, aligned_seq, alignment_score = needleman_wunsch(reference_sequence.seq, seq_to_align)
    print("Needleman-Wunsch Alignment Score:", alignment_score)
    print("Reference Sequence Alignment:", aligned_ref_seq)
    print("Aligned Sequence:", aligned_seq)